In [1]:
!pip install PyPDF2

  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)


In [2]:
!pip install langchain

In [3]:
!pip install openai

In [4]:
!pip install sentence-transformers

In [5]:
!pip install faiss-cpu

In [7]:
import os
from PyPDF2 import PdfReader
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

# OpenAI API Key 설정
os.environ["OPENAI_API_KEY"] = "sk-"  # 실제 API 키 입력


In [8]:
# PDF에서 텍스트 추출
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

In [14]:
# 텍스트를 청크 단위로 분할
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n"],  # 문자열 이스케이프 문제 수정
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

In [15]:
# 텍스트 청크를 벡터 스토어로 변환
def get_vectorstore(text_chunks):
    embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore


In [16]:
# 대화 체인 생성
def get_conversation_chain(vectorstore):
    memory = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True, window_size=10)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo'),
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain

In [17]:
# 사용자로부터 PDF 파일 경로 입력받기
pdf_path = input("PDF 파일 경로를 입력하세요: ").strip()

# PDF 파일 경로 리스트로 만들기
pdf_docs = [pdf_path]

In [18]:
# PDF 텍스트 추출 및 처리
raw_text = get_pdf_text(pdf_docs)
text_chunks = get_text_chunks(raw_text)
vectorstore = get_vectorstore(text_chunks)
conversation_chain = get_conversation_chain(vectorstore)

c:\Users\j0708\Desktop\startup0711-main\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\j0708\Desktop\startup0711-main\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\j0708\Desktop\startup0711-main\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3

In [19]:
# 사용자 질문 입력 및 응답
user_query = input("PDF 내용에 대한 질문을 입력하세요: ").strip()
result = conversation_chain({
    "question": user_query,
    "chat_history": []
})
response = result["answer"]
print("Assistant:", response)

c:\Users\j0708\Desktop\startup0711-main\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Assistant: 프롬프트를 잘 하는 방법은 다음과 같습니다:
1. 명확하고 구체적으로 작성하기
2. 맥락 제공하기
3. 간결하게 작성하기
4. 단계적으로 질문하기
5. 피드백 제공하기
6. 예시 포함하기
7. 열린 질문과 닫힌 질문 구분하기

이러한 방법을 활용하면 AI가 원하는 답변을 더 정확하게 제공할 수 있습니다.
